In [1]:
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np
from pathlib import Path

FOLDER_RAW_ORIG = '../data/raw/pipistrel/2018-10-07_Bodensee_chronologisch/'
FOLDER_SCALED = '../data/processed/pipistrel/scaled-200/'
SCALE_PERCENT = 5
IMAGES_PATH = Path(FOLDER_SCALED)

# Utility functions
def display_image(image_name=None, parent="../data/processed/pipistrel/scaled-200/"):
    image = (Path(parent) / image_name).as_posix()
    plt.imshow(cv2.imread(image))
def get_image_shape(image_name=None, parent="../data/processed/pipistrel/scaled-200/"):
    image = (Path(parent) / image_name).as_posix()
    return cv2.imread(image).shape
def get_image_from_name(image_name=None, parent="../data/processed/pipistrel/scaled-200/"):
    image = (Path(parent) / image_name).as_posix()
    return cv2.imread(image)
def update_mask(mask, bbox_data):
    x1, y1, x2, y2 = bbox_data
    # Remember the image axis are inverted!!! (Y, X)
    mask[y1:y2, x1:x2] = 1
    return mask

def get_df_training_masks(df, scale_percent=5, images_path=None):
    df['boat'] = (df['class'] == 'boat').astype(np.int64)
    df.drop(columns=['class', 'trackid'], inplace=True)
    # Get number of boats per image
    df_nboats = df.groupby('filename')['boat'].sum()
    df_nboats = df_nboats.rename('n_boats')
    # Add number of boats col
    df = df.join(df_nboats, on='filename')
    # Separate boat and no-boat
    df_no_boat = df[df['n_boats']==0]
    # Keep one record per image
    df_no_boat = df_no_boat.groupby('filename', as_index=False)['n_boats'].first()
    # Now boat images
    # Some records have "nature" bboxes that we don't want (boat==0 & n_boat==1)
    df_boat = df[(df['n_boats'] > 0) & (df['boat'] > 0)]
    # Rescale bboxes according to the given scale percentage
    df_boat.loc[:, ['xmin', 'ymin', 'xmax', 'ymax']] = (
        df_boat.loc[:, ['xmin', 'ymin', 'xmax', 'ymax']] * scale_percent / 100
    ).round(0).astype(np.int64)
    # Get one sample image
    pip_images_path = images_path
    image = [x for x in pip_images_path.glob('*.jpg')][0]
    # Create masks for boats
    size=get_image_shape(image.name, parent=images_path.as_posix())[:2]
    print(f'Image size: {size}')
    mask = {}
    for name, grp in tqdm(df_boat.groupby('filename')):
    #     print(i, grp)
        mask_tmp = np.zeros(size)
    #     print(mask_tmp.shape)
        for row in grp.itertuples():
    #         print(row)
            bbox_data = (row.xmin, row.ymin, row.xmax, row.ymax)
    #         print(bbox_data)
            mask_tmp = update_mask(mask_tmp, bbox_data)
        mask[name] = mask_tmp
    # Put mask in Series
    s_mask = pd.Series(mask, name='mask')
    # Create DF with masks
    df_boat = df_boat[['filename', 'n_boats']].groupby(
        'filename',
        as_index=False
    ).first().join(
        s_mask,
        on='filename',
        how='left'
    )
    # Add empty masks to no-boat images
    df_no_boat['mask'] = df_no_boat.apply(lambda x: np.zeros(size), axis=1)
    return pd.concat((df_boat, df_no_boat))

In [26]:
# Training labels
df = pd.read_csv('../data/raw/pipistrel/labelsTrain.csv')
df_train = get_df_training_masks(df, scale_percent=SCALE_PERCENT, images_path=IMAGES_PATH)

 33%|███▎      | 155/472 [00:00<00:00, 1549.62it/s]

Image size: (123, 164)


100%|██████████| 472/472 [00:00<00:00, 1607.00it/s]


In [27]:
df_train.head()

,filename,n_boats,mask
0,Pipstrel-Virus_Bodensee_2018-02-13_15-41-05.jpg,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Pipstrel-Virus_Bodensee_2018-02-13_15-41-06.jpg,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Pipstrel-Virus_Bodensee_2018-02-13_15-41-07.jpg,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,Pipstrel-Virus_Bodensee_2018-02-13_15-41-09.jpg,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Pipstrel-Virus_Bodensee_2018-02-13_15-41-10.jpg,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [28]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 821 entries, 0 to 348
Data columns (total 3 columns):
filename    821 non-null object
n_boats     821 non-null int64
mask        821 non-null object
dtypes: int64(1), object(2)
memory usage: 25.7+ KB
